## $__str__$函数

$__str__$函数 
用于处理打印实例本身的时候的输出内容，如果没有覆写该函数，则默认输出一个对应名称和内存地址。

In [4]:
class Student(object):
    def __init__(self,name):
        self._name = name
print(Student("Tom"))

如何让输出的结果可读性更高一点呢？我么可以覆写$__str__$函数，例如：

In [7]:
class Student(object):
    def __init__(self, name):
        self._name = name
    def __str__(self):
        return "I'm a student ,named %s" % self._name
print(Student("Charlie"))

I'm a student ,named Charlie


我们将str()函数作用于该对象的时候，其实是调用了该对象的__str__函数

## $__repr__$函数

$__repr__$也是将对象序列化，但是$__repr__$更多的是给python编译器看的。$__str__$更多的是可读性(readable)。
我们将repr()函数作用于摸某一个对象的时候，调用的其实就是该函数的$__repr__$函数。

与repr()成对的是eval()函数。eval()函数是将序列化后的对象重新转为对象。前提是该对象实现了$__repr__$函数。

In [8]:
item = [1,2,3]
repr(item)

'[1, 2, 3]'

In [9]:
other_item = eval(repr(item))
other_item[1]

2

## $__iter__$函数

我们经常对list或者tuples使用for..in...来迭代，那是list继承自Iterable,Iterable实现了$__iter__$函数，要想将一
自定义的对象编程一个可迭代的对象，那么必须要实现两个方法：$__iter__$和$__next__$$.
$__iter__$函数返回一个对象，迭代的时候则会不断地调用next函数拿到下一个值，直到捕获到StopIteration停止。

In [43]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0,1
    def __iter__(self):
        return self
    def __next__(self):
        self.a,self.b = self.b,self.a + self.b
        if self.a > 100:
            raise StopIteration
        return self.a
for i in Fib():
    print(i)

1
1
2
3
5
8
13
21
34
55
89


## $__getitem__$函数

上面通过实现$__iter__$函数实现对象的迭代。
那么怎么实现对象按下标去除元素呢。

In [13]:
class Mylist(object):
    def __init__(self, *args):
        self.numbers = args
    def __getitem__(self, item):
        return self.numbers[item]
my_list = Mylist(1,2,3,4,5,6,3)
print(my_list[2])

3


当然列表还有切片功能,例如：my_list[1:3],此时函数获取的并不是int而是slice对象。

In [18]:
class Mylist(object):
    def __init__(self, *args):
        self.numbers = args
    def __getitem__(self, item):
        if isinstance(item,int):
            return self.numbers[item]
        elif isinstance(item,slice):
            start = item.start if item.start is not None else 0
            stop = item.stop if item.stop is not None else len(self.numbers)
            return self.numbers[start:stop]
my_list = Mylist(1,2,3,4,5,6,7)
print(my_list[2:5])

(3, 4, 5)


上面代码实现了切片功能，但是没有考虑负数

In [19]:
class Mylist(object):
    def __init__(self, *args):
        self.numbers = args
    def __getitem__(self, item):
        if isinstance(item,int):
            return self.numbers[item]
        elif isinstance(item,slice):
            start = item.start if item.start is not None else 0
            stop = item.stop if item.stop is not None else len(self.numbers)
            length = len(self.numbers)
            start = start + length + 1 if start < 0 else start
            stop = stop + length + 1 if stop < 0 else stop
            return self.numbers[start:stop]
my_list = Mylist(1,2,3,4,5,6,7)
print(my_list[1:-1])

(2, 3, 4, 5, 6, 7)


## $__getattr__$函数

在调用某个对象不存在的属性或者方法的时候，会跑出一个一个AttributeError错误。
但是如果我们实现了类中的魔术方法$__getattr__$,那么在调用不存在的属性或者方法的时候，就会调用该魔术方法

In [21]:
class Apple(object):
    def __getattr__(self, item):
        if item == "attr1":
            return "print"
        if item == "method1":
            return lambda x: "hello %s" % x
apple = Apple()
print(apple.attar1)
print(apple.method1)

None
<function Apple.__getattr__.<locals>.<lambda> at 0x0000000006C4B378>


$__getattr__$函数一个重要的适用场景就是实现链式调用，例如我们在调用某一个api的时候：

GET users/articles/index

那么我们就希望我们的代码可以实现`Api.users.articles.index这么调用。
思考一下，要实现链式调用，最重要的就是每一个调用都是返回一个实例～～。

In [29]:
class Api(object):
    def __init__(self, path=''):
        self._path = path
    def __getattr__(self, name):
        print("%s/%s" % (self._path, name))
        return Api("%s/%s" % (self._path, name))
    def post(self):
        print(self._path)
api = Api()
api.user.articles.index.post()

/user
/user/articles
/user/articles/index
/user/articles/index


In [28]:
class Apple(object):
    def __call__(self, *args, **kwargs):
        return args


apple = Apple()
print(apple("yes", "no"))

('yes', 'no')


In [26]:
class Api(object):
    def __init__(self, path=''):
        self._path = path
    def __getattr__(self, name):
        return Api("%s/%s" % (self._path, name))
    def __call__(self, args):
        self._path = "%s/%s" % (self._path, args)
        return Api(self._path)
    def post(self):
        print(self._path)
api = Api()
api.user("articles").index.post()

/user/articles/index


In [41]:
import time
import wrapt 
def decorator(eps):
    @wrapt.decorator
    def wrapper(func, instance, args, kwargs):
        t = time.time()
        ans = func(*args ,**kwargs)
        t = time.time() - t
        if t > eps:print("Slow!")
        else:print("Fast!")
        return ans,t
    return wrapper
@decorator(0.01)
def func1():
    for _ in range(10**6):
        x = 0
    return "Done ,it costs:"
@decorator(0.07)
def func2():
    for _ in range(10**6):
        x = 0
    return "Done,it costs:"
print(func1())
print(func2())

Slow!
('Done ,it costs:', 0.06800413131713867)
Fast!
('Done,it costs:', 0.05500292778015137)
